In [3]:
# -*- coding:utf-8 -*-
import requests
import re
from lxml import html
#抓取详细信息
class OpenLawSpider:
    #页面初始化
    def __init__(self):
        self.headers = {
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
            "Accept-Encoding": "gzip, deflate",
            "Accept-Language": "zh-CN,zh;q=0.9",
            "Cache-Control": "max-age=0",
            "Connection": "keep-alive",
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.86 Safari/537.36"
        }
    # 获取详细
    def getLawDetail(self):
        url = 'http://openlaw.cn/search/judgement/type?causeId=d8347b89678645e1887045b4200e822f'
        host = {'host': 'openlaw.cn',}
        headers = self.headers.copy()
        headers.update(host, )

        # 第一步，获取js文件内容
        ret_origin = requests.get(url, headers=headers)
        cookies = requests.utils.dict_from_cookiejar(ret_origin.cookies)
        print(cookies)
        # 第二步，js代码并还原j_token计算过程，正则匹配window.v
        cmp = re.compile('window.v="(.*)";')
        rst = cmp.findall(ret_origin.text)
        v_token = 'abcdefghijklmnopqrstuvwxyz'
        if len(rst):
            v_token = rst[0]
        j_token = v_token[2:4] + 'n' + v_token[0:1] + 'p' + v_token[4:8] + 'e' + v_token[1:2] + v_token[len(v_token)-17:] + v_token[8:16]
        cookies['j_token'] = j_token
        ret_next = requests.get(url, headers=headers, cookies = cookies)
        response = html.fromstring(ret_next.text)
        items = response.cssselect("div[id=primary] .ht-container .entry-title a")
        for item in items:
            title = item.text_content()
            print(title)

spider = OpenLawSpider()
spider.getLawDetail()

{}


In [26]:
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams
from pdfminer.pdfparser import PDFParser, PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfdevice import PDFDevice

def pdfreader(filename):
    #获得文档对象,以二进制读方式打开
    fp = open("（2017）冀0930刑初86号.pdf", "rb")

    #创建一个与文档关联的分析器
    parser = PDFParser(fp)

    #创建一个pdf文档的对象
    doc = PDFDocument()

    #连接解释器与文档对象
    parser.set_document(doc)
    doc.set_parser(parser)

    #初始化文档,如果文档有密码，写与此。
    doc.initialize("")

    #创建pdf资源管理器
    resource = PDFResourceManager()

    #参数分析器
    laparam = LAParams()

    #创建聚合器
    device = PDFPageAggregator(resource, laparams=laparam)

    #创建pdf页面解释器
    interpreter = PDFPageInterpreter(resource, device)

    #使用文档对象得到页面的集合
    list_text,corpus = [],[]
    for page in doc.get_pages():
        #使用页面解释器读取
        interpreter.process_page(page)
        #使用聚合器来获得内容
        layout = device.get_result()
        for out in layout:
            if hasattr(out, "get_text"):
                line = out.get_text().strip("\n ")
                line_clean = re.sub("\n","",line)
                list_text.append(line_clean)
    final_test = "".join([i for i in list_text if i != ""])
    return final_test